# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests 
import gmaps
import os


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_df = pd.read_csv('../output_data/cities.csv')


# Display sample data
city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Baykit,61.6700,96.3700,-37.50,100,100,2.21,RU,01/10/2023
1,Ushuaia,-54.8000,-68.3000,49.66,81,0,16.11,AR,01/10/2023
2,Pacific Grove,36.6177,-121.9166,55.42,82,100,6.91,US,01/10/2023
3,Sainte-Maxime,43.3000,6.6333,51.01,63,99,6.69,FR,01/10/2023
4,Provideniya,64.3833,-173.3000,6.91,61,20,22.37,RU,01/10/2023
...,...,...,...,...,...,...,...,...,...
525,Quirinópolis,-18.4483,-50.4517,68.29,93,92,4.43,BR,01/10/2023
526,Binucayan,8.2500,125.7167,80.35,93,86,2.93,PH,01/10/2023
527,Bisert',56.8617,59.0523,-6.25,96,100,10.54,RU,01/10/2023
528,Rio Claro,-22.4114,-47.5614,67.77,95,100,3.78,BR,01/10/2023


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key = g_key)

In [5]:
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
weather_df = city_df[(city_df["Max Temp"] >= 70)&
                       (city_df["Max Temp"] <= 80)&
                       (city_df["Wind Speed"] <= 10)&
                       (city_df["Cloudiness"] == 0)].dropna()
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,Barra de Tecoanapa,16.5167,-98.7500,75.96,76,0,4.47,MX,01/10/2023
15,Sur,22.5667,59.5289,75.58,68,0,7.34,OM,01/10/2023
230,Cabo San Lucas,22.8909,-109.9124,70.52,69,0,9.78,MX,01/10/2023
256,Tarabha,20.7333,83.6500,72.00,25,0,2.10,IN,01/10/2023
307,Brownsville,25.9018,-97.4975,71.10,97,0,8.05,US,01/10/2023
311,Yārāda,17.6500,83.2667,77.32,57,0,3.44,IN,01/10/2023
312,Acapulco de Juárez,16.8634,-99.8901,76.82,83,0,0.00,MX,01/10/2023
428,Tura,25.5198,90.2201,70.83,40,0,1.66,IN,01/10/2023
513,Teknāf,20.8624,92.3058,75.52,46,0,5.08,BD,01/10/2023


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = weather_df
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
9,Barra de Tecoanapa,16.5167,-98.7500,75.96,76,0,4.47,MX,01/10/2023,
15,Sur,22.5667,59.5289,75.58,68,0,7.34,OM,01/10/2023,
230,Cabo San Lucas,22.8909,-109.9124,70.52,69,0,9.78,MX,01/10/2023,
256,Tarabha,20.7333,83.6500,72.00,25,0,2.10,IN,01/10/2023,
307,Brownsville,25.9018,-97.4975,71.10,97,0,8.05,US,01/10/2023,
311,Yārāda,17.6500,83.2667,77.32,57,0,3.44,IN,01/10/2023,
312,Acapulco de Juárez,16.8634,-99.8901,76.82,83,0,0.00,MX,01/10/2023,
428,Tura,25.5198,90.2201,70.83,40,0,1.66,IN,01/10/2023,
513,Teknāf,20.8624,92.3058,75.52,46,0,5.08,BD,01/10/2023,


In [10]:
# Create URL and loop to gather information
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

In [11]:


# set up a parameters dictionary
params = {
    "type": "hotel",
    "keyword": "hotel",
    "radius": 5000,
    "key": g_key}

In [12]:
# geocoordinates
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    #change paramaters for location
    params["location"] = f"{lat},{lng}"
    
    response = requests.get(base_url, params=params).json()
    
    # save hotel information
    try:
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
        #hotel_df.append(response['results'][0]['name'])
    except IndexError:
        print("Hotels not found")

Hotels not found


In [13]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
9,Barra de Tecoanapa,16.5167,-98.7500,75.96,76,0,4.47,MX,01/10/2023,Hotel Garapacho
15,Sur,22.5667,59.5289,75.58,68,0,7.34,OM,01/10/2023,Sur Grand Hotel
230,Cabo San Lucas,22.8909,-109.9124,70.52,69,0,9.78,MX,01/10/2023,Solmar Resort
256,Tarabha,20.7333,83.6500,72.00,25,0,2.10,IN,01/10/2023,
307,Brownsville,25.9018,-97.4975,71.10,97,0,8.05,US,01/10/2023,Boca Chica Inn & Suites
311,Yārāda,17.6500,83.2667,77.32,57,0,3.44,IN,01/10/2023,Nosina residency
312,Acapulco de Juárez,16.8634,-99.8901,76.82,83,0,0.00,MX,01/10/2023,Hotel Krystal Beach Acapulco
428,Tura,25.5198,90.2201,70.83,40,0,1.66,IN,01/10/2023,Hotel Polo Orchid
513,Teknāf,20.8624,92.3058,75.52,46,0,5.08,BD,01/10/2023,Central Resort Ltd.


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add markers on to heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))